**Installing and importing required packages**

In [1]:
#!pip install -U libsvm-official
from libsvm.svmutil import *
import numpy as np
import scipy.io

**Reading input parameters**

In [2]:
train_data = scipy.io.loadmat('/content/trainData.mat') 
test_data = scipy.io.loadmat('/content/testData.mat')

**Extracting features from dataset**

In [3]:
x1_train = train_data['X1']
x2_train = train_data['X2']
x3_train = train_data['X3']
y_train = train_data['Y'].flatten()

x1_test = test_data['X1']
x2_test = test_data['X2']
x3_test = test_data['X3']
y_test = test_data['Y'].flatten()

**Step 0 - Classification by individual features.**

**Part1**

In [4]:
x1_Step0_Part1 = svm_problem(y_train, x1_train)
x2_Step0_Part1 = svm_problem(y_train, x2_train)
x3_Step0_Part1 = svm_problem(y_train, x3_train)

#Creating the tarin model hk for each Xk
def train_model(problem, parameters):
    print(f"Training Started.")
    model = svm_train(problem, parameters)
    print(f"Training Completed.")
    print(f"*****************************************************")
    return model

parameters = svm_parameter("-c 10 -t 0 -q") 
# Naming convention of model: model{k}_Step{Step}_{Sub Problem if any}
model1_Step0_Part1 = train_model(x1_Step0_Part1, parameters)
model2_Step0_Part1 = train_model(x2_Step0_Part1, parameters)
model3_Step0_Part1 = train_model(x3_Step0_Part1, parameters)

# Performing predictions on the test dataset for the above created models.
m1_label_Step0_Part1, m1_accuracy_Step0_Part1, m1_val_Step0_Part1 = svm_predict(y_test, x1_test, model1_Step0_Part1)
m2_label_Step0_Part1, m2_accuracy_Step0_Part1, m2_val_Step0_Part1 = svm_predict(y_test, x2_test, model2_Step0_Part1)
m3_label_Step0_Part1, m3_accuracy_Step0_Part1, m3_val_Step0_Part1 = svm_predict(y_test, x3_test, model3_Step0_Part1)

# Printing the accuracies for each model by rounding off to 2 digits
print(f"========================Printing the accuracies for Step 0 - Part1========================")
print(f"Accuracy for X1 Dataset: {m1_accuracy_Step0_Part1[0]: .2f}%.")
print(f"Accuracy for X2 Dataset: {m2_accuracy_Step0_Part1[0]: .2f}%.")
print(f"Accuracy for X3 Dataset: {m3_accuracy_Step0_Part1[0]: .2f}%.\n")

Training Started.
Training Completed.
*****************************************************
Training Started.
Training Completed.
*****************************************************
Training Started.
Training Completed.
*****************************************************
Accuracy = 10.7807% (203/1883) (classification)
Accuracy = 16.6755% (314/1883) (classification)
Accuracy = 8.92193% (168/1883) (classification)
========================Printing the accuracies for Step 0 - Part1========================
Accuracy for X1 Dataset:  10.78%.
Accuracy for X2 Dataset:  16.68%.
Accuracy for X3 Dataset:  8.92%.



**Part2**

In [5]:
#Here we are modifying the parameters from part1 to return the probabilities
parameters = svm_parameter("-c 10 -t 0 -b 1 -q") 

#Here We can using the same problem that we created in the part1 to define our models.
model1_Step0_Part2 = train_model(x1_Step0_Part1, parameters)
model2_Step0_Part2 = train_model(x2_Step0_Part1, parameters)
model3_Step0_Part2 = train_model(x3_Step0_Part1, parameters)

# Performing predictions on the test dataset for the above created models.
m1_label_Step0_Part2, m1_accuracy_Step0_Part2, m1_val_Step0_Part2 = svm_predict(y_test, x1_test, model1_Step0_Part2,"-b 1")
m2_label_Step0_Part2, m2_accuracy_Step0_Part2, m2_val_Step0_Part2 = svm_predict(y_test, x2_test, model2_Step0_Part2,"-b 1")
m3_label_Step0_Part2, m3_accuracy_Step0_Part2, m3_val_Step0_Part2 = svm_predict(y_test, x3_test, model3_Step0_Part2,"-b 1")

# Printing the accuracies for each model
print(f"**********************************Printing the accuracies for Step 0 - Part2**********************************")
print(f"Accuracy for X1 Dataset: {m1_accuracy_Step0_Part2[0]: .2f}%.")
print(f"Accuracy for X2 Dataset: {m2_accuracy_Step0_Part2[0]: .2f}%.")
print(f"Accuracy for X3 Dataset: {m3_accuracy_Step0_Part2[0]: .2f}%.\n")

Training Started.
Training Completed.
*****************************************************
Training Started.
Training Completed.
*****************************************************
Training Started.
Training Completed.
*****************************************************
Accuracy = 27.881% (525/1883) (classification)
Accuracy = 27.5093% (518/1883) (classification)
Accuracy = 28.7839% (542/1883) (classification)
**********************************Printing the accuracies for Step 0 - Part2**********************************
Accuracy for X1 Dataset:  27.88%.
Accuracy for X2 Dataset:  27.51%.
Accuracy for X3 Dataset:  28.78%.



**Step 1 - Feature combination by fusion of classifiers**

In [8]:
sample_prob = []
sample_label = []
for prob_x1, prob_x2, prob_x3 in zip(m1_val_Step0_Part2, m2_val_Step0_Part2, m3_val_Step0_Part2):
    mean = np.array(prob_x1) + np.array(prob_x2) + np.array(prob_x3)
    mean = mean / 3.0
    sample_prob.append(list(mean))
    sample_label.append(np.where(mean == max(mean))[0][0] + 1)

correct = (np.array(y_test) == np.array(sample_label))
accuracy = correct.sum() / correct.size
print(f"**********************************Printing the accuracies for Step 1**********************************")
print(f"Accuracy for fusion of classifiers: {accuracy * 100:.2f}%")

print(f"**********************************Printing the accuracies for Step 0 Part 2**********************************")
print(f"Accuracy for X1 of Step0 Part2: {m1_accuracy_Step0_Part2[0]:.2f}%.")
print(f"Accuracy for X2 of Step0 Part2: {m2_accuracy_Step0_Part2[0]:.2f}%.")
print(f"Accuracy for X3 of Step0 Part2: {m3_accuracy_Step0_Part2[0]:.2f}%.")

**********************************Printing the accuracies for Step 1**********************************
Accuracy for fusion of classifiers: 44.72%
**********************************Printing the accuracies for Step 0 Part 2**********************************
Accuracy for X1 of Step0 Part2: 27.88%.
Accuracy for X2 of Step0 Part2: 27.51%.
Accuracy for X3 of Step0 Part2: 28.78%.


**Step 2 - Feature combination by simple concatenation**

In [7]:

# concatening all the feature dataset into one
x_train_concat = np.concatenate((x1_train, x2_train, x3_train), axis=1)
x_test_concat = np.concatenate((x1_test, x2_test, x3_test), axis=1)

parameters = svm_parameter("-c 10 -t 0 -q")
x_Step2 = svm_problem(y_train, x_train_concat)

model_Step2 = train_model(x_Step2, parameters)

m_label_Step2, m_accuracy_Step2, m_val_Step2 = svm_predict(y_test, x_test_concat, model_Step2)

print(f"**********************************Printing the accuracies for Step 2**********************************")
print(f"Accuracy for X combined: {m_accuracy_Step2[0]: .2f}%.")

print(f"**********************************Printing the accuracies for Step 0 Part1 **********************************")
print(f"Accuracy for X1 of Step0 Part1: {m1_accuracy_Step0_Part1[0]: .2f}%.")
print(f"Accuracy for X2 of Step0 Part1: {m2_accuracy_Step0_Part1[0]: .2f}%.")
print(f"Accuracy for X3 of Step0 Part1: {m3_accuracy_Step0_Part1[0]: .2f}%.\n")

Training Started.
Training Completed.
*****************************************************
Accuracy = 37.0685% (698/1883) (classification)
**********************************Printing the accuracies for Step 2**********************************
Accuracy for X combined:  37.07%.
**********************************Printing the accuracies for Step 0 Part1 **********************************
Accuracy for X1 of Step0 Part1:  10.78%.
Accuracy for X2 of Step0 Part1:  16.68%.
Accuracy for X3 of Step0 Part1:  8.92%.

